In [1]:
import pandas as pd

In [2]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('Użytkownik przesłał plik "{name}" o rozmiarze {length} bajtów'.format(
      name=fn, length=len(uploaded[fn])))


Saving matches-prepared.csv to matches-prepared.csv
Użytkownik przesłał plik "matches-prepared.csv" o rozmiarze 36695464 bajtów


In [3]:
df = pd.read_csv('matches-prepared.csv')

In [4]:
display(df.head())

,Division,MatchDate,HomeTeam,AwayTeam,HomeElo,AwayElo,Form3Home,Form5Home,Form3Away,Form5Away,...,OddAway,C_LTH,C_LTA,C_VHD,C_VAD,C_HTB,C_PHB,match_result,total_goals,over25_binary
0,F1,2000-07-28,Marseille,Troyes,1686.34,1586.57,0.0,0.0,0.0,0.0,...,4.3,NaN,NaN,NaN,NaN,NaN,NaN,Win,4.0,1
1,F1,2000-07-28,Paris SG,Strasbourg,1714.89,1642.51,0.0,0.0,0.0,0.0,...,4.6,NaN,NaN,NaN,NaN,NaN,NaN,Win,4.0,1
2,F2,2000-07-28,Wasquehal,Nancy,1465.08,1633.80,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Loss,1.0,0
3,F1,2000-07-29,Auxerre,Sedan,1635.58,1624.22,0.0,0.0,0.0,0.0,...,3.9,NaN,NaN,NaN,NaN,NaN,NaN,Loss,1.0,0
4,F1,2000-07-29,Bordeaux,Metz,1734.34,1673.11,0.0,0.0,0.0,0.0,...,4.3,NaN,NaN,NaN,NaN,NaN,NaN,Draw,2.0,0


### 1. Tworzenie zmiennej docelowej `high_score`

In [5]:
# Obliczanie sumy bramek
df['TotalGoals'] = df['FTHome'] + df['FTAway']

# Tworzenie zmiennej docelowej high_score
# 'yes' jeśli suma bramek > 2, 'no' w przeciwnym razie
df['high_score'] = df['TotalGoals'].apply(lambda x: 'yes' if x > 2 else 'no')

# Sprawdzenie rozkładu nowej zmiennej docelowej
print(df['high_score'].value_counts())
print("\n")

display(df[['FTHome', 'FTAway', 'TotalGoals', 'high_score']].head())

high_score
no     117279
yes    113275
Name: count, dtype: int64




,FTHome,FTAway,TotalGoals,high_score
0,3.0,1.0,4.0,yes
1,3.0,1.0,4.0,yes
2,0.0,1.0,1.0,no
3,0.0,1.0,1.0,no
4,1.0,1.0,2.0,no


### 2. Wybór cech i przygotowanie do kodowania/skalowania

In [18]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Zdefiniowanie cech (X) i zmiennej docelowej (y)
# Wyłączyłem kolumny, które nie są cechami lub są zmiennymi do stworzenia targetu, w tym te powodujące możliwy wyciek danych
X = df.drop(columns=['Division', 'MatchDate', 'FTHome', 'FTAway', 'TotalGoals', 'high_score', 'total_goals', 'over25_binary'])
y = df['high_score']

# Automatyczne wykrywanie cech numerycznych i kategorycznych
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

print(f"Cechy numeryczne: {list(numerical_features)}")
print(f"Cechy kategoryczne: {list(categorical_features)}")

Cechy numeryczne: ['HomeElo', 'AwayElo', 'Form3Home', 'Form5Home', 'Form3Away', 'Form5Away', 'HTHome', 'HTAway', 'HomeShots', 'AwayShots', 'HomeTarget', 'AwayTarget', 'HomeFouls', 'AwayFouls', 'HomeCorners', 'AwayCorners', 'HomeYellow', 'AwayYellow', 'HomeRed', 'AwayRed', 'OddHome', 'OddDraw', 'OddAway', 'C_LTH', 'C_LTA', 'C_VHD', 'C_VAD', 'C_HTB', 'C_PHB']
Cechy kategoryczne: ['HomeTeam', 'AwayTeam', 'FTResult', 'HTResult', 'match_result']


### 3. Dodatkowe przetwarzanie: kodowanie i skalowanie

In [19]:
from sklearn.impute import SimpleImputer

numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Tworzenie preprocesora za pomocą ColumnTransformer
# Stosuję StandardScaler do cech numerycznych i OneHotEncoder do cech kategorycznych
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Zastosowanie preprocesora do danych
X_processed = preprocessor.fit_transform(X)

print(f"Kształt przetworzonego zbioru cech: {X_processed.shape}")

# Podział danych na zbiory treningowy i testowy
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42, stratify=y)

print(f"Kształt zbioru treningowego (X_train): {X_train.shape}")
print(f"Kształt zbioru testowego (X_test): {X_test.shape}")
print(f"Kształt zbioru treningowego (y_train): {y_train.shape}")
print(f"Kształt zbioru testowego (y_test): {y_test.shape}")

Kształt przetworzonego zbioru cech: (230554, 2458)
Kształt zbioru treningowego (X_train): (184443, 2458)
Kształt zbioru testowego (X_test): (46111, 2458)
Kształt zbioru treningowego (y_train): (184443,)
Kształt zbioru testowego (y_test): (46111,)


## Wytrenowanie modelu regresji logistycznej



In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

print("Zaimportowano moduły `LogisticRegression` i `accuracy_score`.")

Zaimportowano moduły `LogisticRegression` i `accuracy_score`.


In [10]:
model_lr = LogisticRegression(solver='liblinear', random_state=42)

# Trenowanie modelu
model_lr.fit(X_train, y_train)

# Predykcja na danych testowych
y_pred_lr = model_lr.predict(X_test)

# Obliczanie dokładności
accuracy_lr = accuracy_score(y_test, y_pred_lr)
print(f"Dokładność modelu regresji logistycznej: {accuracy_lr:.4f}")

Dokładność modelu regresji logistycznej: 0.7437


## Wytrenowanie modelu maszyny wektorów nośnych (SVM)



In [11]:
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

print("Zaimportowano moduły `LinearSVC` i `accuracy_score`.")

Zaimportowano moduły `LinearSVC` i `accuracy_score`.


In [12]:
model_svm = LinearSVC(random_state=42, dual=False)

# Trenowanie modelu
model_svm.fit(X_train, y_train)

# Predykcja na danych testowych
y_pred_svm = model_svm.predict(X_test)

# Obliczanie dokładności
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"Dokładność modelu maszyny wektorów nośnych (LinearSVC): {accuracy_svm:.4f}")

Dokładność modelu maszyny wektorów nośnych (LinearSVC): 0.7431


## Wytrenowanie modelu lasu losowego (Random Forest)



In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

print("Zaimportowano moduły `RandomForestClassifier` i `accuracy_score`.")

Zaimportowano moduły `RandomForestClassifier` i `accuracy_score`.


In [14]:
model_rf = RandomForestClassifier(random_state=42)

# Trenowanie modelu
model_rf.fit(X_train, y_train)

# Predykcja na danych testowych
y_pred_rf = model_rf.predict(X_test)

# Obliczanie dokładności
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print(f"Dokładność modelu lasu losowego: {accuracy_rf:.4f}")

Dokładność modelu lasu losowego: 0.7577


In [15]:
print(f"Dokładność modelu regresji logistycznej: {accuracy_lr:.4f}")
print(f"Dokładność modelu maszyny wektorów nośnych (LinearSVC): {accuracy_svm:.4f}")
print(f"Dokładność modelu lasu losowego: {accuracy_rf:.4f}")

# Porównanie modeli
model_accuracies = {
    "Logistic Regression": accuracy_lr,
    "LinearSVC": accuracy_svm,
    "Random Forest": accuracy_rf
}

best_model = max(model_accuracies, key=model_accuracies.get)
best_accuracy = model_accuracies[best_model]

print(f"\nNajlepszym modelem pod względem dokładności jest: {best_model} z dokładnością: {best_accuracy:.4f}")

Dokładność modelu regresji logistycznej: 0.7437
Dokładność modelu maszyny wektorów nośnych (LinearSVC): 0.7431
Dokładność modelu lasu losowego: 0.7577

Najlepszym modelem pod względem dokładności jest: Random Forest z dokładnością: 0.7577


## Zapis wytrenowanego modelu


In [16]:
import joblib

# Zapisanie najlepszego modelu do pliku
filename = 'high_score_model.pkl'
joblib.dump(model_rf, filename)

print(f"Model '{best_model}' został pomyślnie zapisany do '{filename}'")

Model 'Random Forest' został pomyślnie zapisany do 'high_score_model.pkl'


### Testowanie zapisanego modelu


In [20]:
# Wczytanie modelu z pliku
loaded_model = joblib.load(filename)
print(f"Model został pomyślnie wczytany z '{filename}'")

# Wykonanie predykcji na wczytanym modelu (przykładowo na fragmencie danych testowych)
y_pred_loaded = loaded_model.predict(X_test)

# Sprawdzenie, czy predykcje wczytanego modelu są takie same jak oryginalne
from sklearn.metrics import accuracy_score
accuracy_loaded = accuracy_score(y_test, y_pred_loaded)

print(f"Dokładność wczytanego modelu na danych testowych: {accuracy_loaded:.4f}")

if accuracy_loaded == accuracy_rf:
    print("Wczytany model działa poprawnie i daje identyczne wyniki jak oryginalny.")
else:
    print("Błąd. Dokładność wczytanego modelu różni się od oryginalnego.")

Model został pomyślnie wczytany z 'high_score_model.pkl'
Dokładność wczytanego modelu na danych testowych: 0.7577
Wczytany model działa poprawnie i daje identyczne wyniki jak oryginalny.
